#From Audio to Photoreal Embodiment: Synthesizing Humans in Conversations

In this tutorial, we will walk you through setting up the environment and running the gradio app that will let you drive a photorealistic avatar using your voice.

More useful links: [Arxiv]() | [Code](https://github.com/facebookresearch/audio2photoreal/) | [Project page](https://people.eecs.berkeley.edu/~evonne_ng/projects/audio2photoreal/)

# Environment setup
Simply run through all of the 3 cells below. This will install the proper environment, download assets, and place them in the right places.

In [2]:
# Setup environment and install requirements
!pip install -r scripts/requirements.txt

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'scripts/requirements.txt'


In [ ]:
# download models, rendering assets, and prerequisite models respectively
!wget http://audio2photoreal_models.berkeleyvision.org/PXB184_models.tar
!tar xvf PXB184_models.tar
!rm PXB184_models.tar

!mkdir -p checkpoints/ca_body/data/
!wget https://github.com/facebookresearch/ca_body/releases/download/v0.0.1-alpha/PXB184.tar.gz
!tar xvf PXB184.tar.gz --directory checkpoints/ca_body/data/
!rm PXB184.tar.gz

!wget http://audio2photoreal_models.berkeleyvision.org/asset_models.tar
!tar xvf asset_models.tar
!rm asset_models.tar

--2024-08-14 12:39:35--  http://audio2photoreal_models.berkeleyvision.org/PXB184_models.tar
Resolving audio2photoreal_models.berkeleyvision.org (audio2photoreal_models.berkeleyvision.org)... 128.32.162.150
Connecting to audio2photoreal_models.berkeleyvision.org (audio2photoreal_models.berkeleyvision.org)|128.32.162.150|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1350748160 (1.3G) [application/octet-stream]
Saving to: ‘PXB184_models.tar’

PXB184_models.tar   100%[===================>]   1.26G  52.9MB/s    in 26s     

2024-08-14 12:40:01 (50.3 MB/s) - ‘PXB184_models.tar’ saved [1350748160/1350748160]

checkpoints/diffusion/c1_face/model000155000.pt
checkpoints/diffusion/c1_face/args.json
checkpoints/diffusion/c1_pose/model000340000.pt
checkpoints/diffusion/c1_pose/args.json
checkpoints/guide/c1_pose/args.json
checkpoints/guide/c1_pose/checkpoints/iter-0100000.pt
checkpoints/vq/c1_pose/args.json
checkpoints/vq/c1_pose/net_iter300000.pth
--2024-08-14 12:40:04-

In [ ]:
# install pytorch3d

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61395 sha256=d1ac38f2da84999c3aff9a1dcc6a2b4b59b80f2cd561ff2f1b162e2043240e6c
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=bd2db4670d681ec6f94698c2b6fe2d81307cd2251b03186ba9185b57bde17121
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-utcgdyr7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /

# Run the model
**Important!!** Before you can run the model, there are two things you must fix.


1.   Fix the runtime settings for collections. With python >= 3.10, google collab will complain `ImportError: cannot import name 'Mapping' from 'collections`.
As a result, you *will* manually need to correct the path for collections from `import collections` to `import collections.abc` for all files that the environment complains about. You can just directly click into those files and change the path. See [this post](https://stackoverflow.com/questions/69381312/importerror-cannot-import-name-mapping-from-collections-using-python-3-10) for more details.

2.   Change the demo script to deploy a public link. You will need to go into `audio2photoreal/demo/demo.py` and on line 272, change from `demo.launch(show_api=False)` to `demo.launch(share=True)`

These are all the file paths I had to change:


* /usr/local/lib/python3.10/dist-packages/attrdict/mapping.py
* /usr/local/lib/python3.10/dist-packages/attrdict/mixins.py
* /usr/local/lib/python3.10/dist-packages/attrdict/merge.py
* /usr/local/lib/python3.10/dist-packages/attrdict/default.py
* /content/audio2photoreal/demo/demo.py

If anyone knows how to revert colab to python==3.9 and would like to share that tidbit with me, would greatly appreciate an email ping :)

**👆 Tip!** It takes at least 8-12 seconds of audio before the lip synching starts working better. It's definitely a limitation of our work, but with clips longer than 8 seconds, you end up getting much better results.

After you finish those two changes, you can go ahead and run the below cell. It will return a *public URL* that you can click into.


In [1]:
import torch
torch.cuda.is_available()

True

In [ ]:
!python -m pip install pip==23.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install gradio
!pip install attrdict
!pip install fairseq
!pip install mediapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.0
    Uninstalling tomlkit-0

In [ ]:
!python -m demo.demo

2024-08-14 15:43:12.136742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 15:43:12.157071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 15:43:12.163559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 15:43:12.179085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 15:43:13.431311: W tensorflow/compiler/tf2